# Virtual Assistant

Connecting your chatbot to some source of external data source allows it to be more useful.
For example, linking to a rest API or database allows it to send queries.

The course discusses good practice in allowing users to amend query parameters. `sqllite3`
package is used and the course emphasises the safeguards that the `cursor.execute('SQL string ?', ?params)`
function has, preventing the users to send malicious queries that delete the database etc - SQL injection.

It is discouraged to use f strings or `.format()` for these same reasons.


In [3]:
import pandas as pd
from pyprojroot import here
import os

## Create a Database

In [24]:
from pathlib import Path
import sqlite3 # included in standard library

dbPath = os.path.join(here(), "data", "restaurants.db")
try:
    Path(dbPath).touch()
except:
    pass

In [25]:
con = sqlite3.connect(dbPath)
curs = con.cursor()

## Ingest restaurant Data

In [49]:
rests = pd.read_csv(
    "https://fsa-catalogue2.s3.eu-west-2.amazonaws.com/Approved+establishments+01-09-2022.csv",
    usecols=["TradingName", "Town", "Country", "CompetentAuthority"])


('TradingName', 'Town', 'Country', 'CompetentAuthority')

In [45]:
# create the sql table ready for write
curs.execute("""CREATE TABLE IF NOT EXISTS restaurants ('TradingName', 'Town', 'Country', 'CompetentAuthority');""")

## Write CSV to DataBase

In [50]:
# write the csv to the db
rests.to_sql('restaurants', con, if_exists='append', index = False)
del rests

5409

In [56]:
params = ("Wales", "Food Standards Agency")
curs.execute("SELECT TradingName FROM restaurants WHERE Country = ? AND CompetentAuthority = ?", params)
curs.fetchall()

[('2 Sisters Food Group Limited T/A 2 Sisters Llangefini',),
 ('The Food Technology Centre',),
 ('Preseli Bluestone Meats Limited',),
 ('2 Sisters Food Group Limited T/A 2 Sisters Sandycroft',),
 ('Faccenda Foods Limited (T/A Avara Foods)',),
 ('Dunbia (UK)',),
 ('Llechwedd Meats',),
 ('Cig Eryri - Snowdonia Meats Cyf',),
 ("Daphne's Welsh Lamb Limited",),
 ('Celtica Foods Limited',),
 ('Bwydlyn',),
 ('Universal Foods',),
 ('M K (Wales) Ltd T/A (Maddock Kembery Meats)',),
 ('Weeks Wholesale Meat Company Ltd',),
 ('I & B Jones Ltd',),
 ('William Arthur James (T/A WA James Butchers)',),
 ('Maelor Foods Limited',),
 ('Hugh Phillips Gower Butcher',),
 ('Mid Glamorgan Provisions Ltd',),
 ('Farmers Pantry Ltd',),
 ('Williams Lloyd Williams and Son Limited',),
 ('WJ George (Butchers) Ltd',),
 ('Huw Evans & Morag Evans (T/A Cig Oen Caron)',),
 ('Direct Meat Supplies Ltd',),
 ('Conwy Valley Meats Limited',),
 ('David Thomas James Thomas Stephen Thomas & Simeon Thomas (T/A D & J Thomas)',),
 ('C